In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os

import torch
print(torch.__version__)

# Any results you write to the current directory are saved as output.

0.3.0.post4


In [25]:
# http://pytorch.org/
from os.path import exists
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'
! conda install pytorch torchvision -c pytorch

# !pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.4.1-{platform}-linux_x86_64.whl torchvision
import torch

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.4.11
  latest version: 4.5.11

Please update conda by running

    $ conda update -n base conda



# All requested packages already installed.



# Introduction to Tensors and Ops in PyTorch

Table of Contents:
- Basics
- Convolution Arithmetic



The fundamental unit in PyTorch is a Tensor. 

In [43]:
x = torch.rand(5, 3)
print(x, type(x))


 0.6428  0.7201  0.2840
 0.0313  0.4677  0.4223
 0.4246  0.2157  0.8328
 0.8905  0.3039  0.5638
 0.7764  0.8109  0.3469
[torch.FloatTensor of size 5x3]
 <class 'torch.FloatTensor'>


'0.3.0.post4'

In [40]:
x = torch.zeros(5, 3)
print(x, type(x))


 0  0  0
 0  0  0
 0  0  0
 0  0  0
 0  0  0
[torch.FloatTensor of size 5x3]
 <class 'torch.FloatTensor'>


In [28]:
x.size()

torch.Size([5, 3])

In [29]:
y = torch.rand(5, 3)
y


 0.1330  0.4241  0.0304
 0.2935  0.9090  0.3644
 0.6394  0.6521  0.4100
 0.4099  0.4805  0.7992
 0.9938  0.3786  0.7293
[torch.FloatTensor of size 5x3]

In [30]:
x + y


 0.1330  0.4241  0.0304
 0.2935  0.9090  0.3644
 0.6394  0.6521  0.4100
 0.4099  0.4805  0.7992
 0.9938  0.3786  0.7293
[torch.FloatTensor of size 5x3]

In [31]:
torch.add(x, y)


 0.1330  0.4241  0.0304
 0.2935  0.9090  0.3644
 0.6394  0.6521  0.4100
 0.4099  0.4805  0.7992
 0.9938  0.3786  0.7293
[torch.FloatTensor of size 5x3]

### Inplace operations
Sometimes, we would like to operations to the existing variable without creating a new one. In such cases, the inplace operations' operator is super handy. For most of the functions, the inplace operator would exist and is usually available as operator followed by an underscore **(operator_)** . For example, **.add** -> **.add_** , **.mul** -> **.mul_** and etc.

In [32]:
x.add_(y)


 0.1330  0.4241  0.0304
 0.2935  0.9090  0.3644
 0.6394  0.6521  0.4100
 0.4099  0.4805  0.7992
 0.9938  0.3786  0.7293
[torch.FloatTensor of size 5x3]

In [33]:
x


 0.1330  0.4241  0.0304
 0.2935  0.9090  0.3644
 0.6394  0.6521  0.4100
 0.4099  0.4805  0.7992
 0.9938  0.3786  0.7293
[torch.FloatTensor of size 5x3]

There are 100+ operations that are possible using PyTorch tensors and a comprehensive list is available here. Do check it out - https://pytorch.org/docs/stable/torch.html

### Numpy Arrays <=> PyTorch Tensors
It's a very common practice to convert numpy arrays into torch tensors and vice-versa and it's super simple do this operation. What's handy is that when this conversion is executed, a new memory allocation is avoided and both the variables share the same memory location, thus being memory efficient. Deep Copy (fresh copy) is also useful, whenever required.

In [34]:
x = np.random.rand(2,3)
x

array([[ 0.32361608,  0.17834695,  0.33969199],
       [ 0.62370845,  0.43763426,  0.14500712]])

In [35]:
y = torch.from_numpy(x)
y


 0.3236  0.1783  0.3397
 0.6237  0.4376  0.1450
[torch.DoubleTensor of size 2x3]

In [36]:
z = y.numpy()
z

array([[ 0.32361608,  0.17834695,  0.33969199],
       [ 0.62370845,  0.43763426,  0.14500712]])

### Exercise

In [37]:
y.add_(1)


 1.3236  1.1783  1.3397
 1.6237  1.4376  1.1450
[torch.DoubleTensor of size 2x3]

In [38]:
x

array([[ 1.32361608,  1.17834695,  1.33969199],
       [ 1.62370845,  1.43763426,  1.14500712]])

### Tensors on CPU and GPU
Moving tensors from CPU and GPU is as trivial as **.cuda()** and **.cpu()**. Observe the types of the tensors once you do this. These small tidbits are important while debugging large PyTorch programs.

First, check if GPU is available or not through code. Here are other handy functions to obtain some metadata.

In [53]:
torch.cuda.is_available(), #torch.cuda.current_device()

(False,)

In [18]:
torch.version.cuda, torch.cuda.device_count()

('9.2.148', 1)

In [19]:
x = torch.rand(2,3)
x

tensor([[0.6259, 0.5719, 0.7578],
        [0.3048, 0.7027, 0.3277]])

In [20]:
x.dtype

torch.float32

In [0]:
y = x.cuda()

In [22]:
y

tensor([[0.6259, 0.5719, 0.7578],
        [0.3048, 0.7027, 0.3277]], device='cuda:0')

In [23]:
x.dtype

torch.float32

In [24]:
x.to('cuda')

tensor([[0.6259, 0.5719, 0.7578],
        [0.3048, 0.7027, 0.3277]], device='cuda:0')

In [25]:
y

tensor([[0.6259, 0.5719, 0.7578],
        [0.3048, 0.7027, 0.3277]], device='cuda:0')

In [26]:
y.cpu()

tensor([[0.6259, 0.5719, 0.7578],
        [0.3048, 0.7027, 0.3277]])

## Variables 

In [45]:
from torch.autograd import Variable

In [46]:
x = Variable(torch.rand(2, 3), requires_grad=True)

In [47]:
x.grad

##  Differentiation

In [54]:
x = torch.Tensor([3])
x = Variable(x, requires_grad=True)
y = 2 * x ** 2 + 3

In [55]:
y

Variable containing:
 21
[torch.FloatTensor of size 1]

In [56]:
y.backward(torch.Tensor([2]))

In [57]:
x.grad

Variable containing:
 24
[torch.FloatTensor of size 1]